In [ ]:
import ipywidgets as widgets
from ruamel.yaml import YAML
from dataclasses import dataclass
from IPython.display import display
import os
import sys
from uuid import uuid4
from functools import partial, partialmethod
#print(widgets.__version__)
import collections
from ipyfilechooser import FileChooser
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


def partialclass(cls, *args, **kwds):

    class NewCls(cls):
        __init__ = partialmethod(cls.__init__, *args, **kwds)

    return NewCls


if __name__ == '__main__':
    Config = partialclass(collections.defaultdict, list)
    assert isinstance(Config(), Config)
yaml = YAML()

In [ ]:
config_path = os.getenv("CONFIG_FILE_PATH", f"config/config.yaml")
project_path = os.getenv("PROJECT_PATH", f"/home/fxlyonnet/repos/shape-ce/examples/test")

#config_path = f"config/test.yaml"

In [ ]:
def nested_dict_get(data: dict, position : []):
    try:
        subdata = data
        for pos in position:
            subdata = subdata[pos]
        
        return subdata
    except:
        return None

def nested_dict_set(data: dict, position : [], value):    
    subdata = data
    for pos in position[:-1]:
        subdata = subdata[pos]
    
    subdata[position[-1]] = value

class FileMV():
    def __init__(self, value, base_path,  sandbox=True, **kwargs):
        self.base_path = base_path
        fullpath = base_path if value is None else os.path.join(base_path,value)
        if os.path.isdir(fullpath):
            curdir = fullpath
            curfile = ""
        else:
            curdir = os.path.dirname(fullpath)
            curfile = os.path.basename(fullpath)
        self._widget = FileChooser(path=curdir,filename=curfile, select_default=True, default_path=base_path,sandbox_path= base_path if sandbox else None, **kwargs)    
            
    @property
    def value(self):
        return os.path.relpath(self._widget.selected, self.base_path)
    
    @property
    def widget(self):
        return self._widget

class BoxMV():
    def __init__(self, value, **kwargs):
        self._widget = widgets.Box()
    @property
    def value(self):
        return None
    
    @property
    def widget(self):
        return self._widget

class EnumMV():
    def __init__(self, value, listMV, **kwargs):
        val = value if value in listMV.keys else None
        self._widget = widgets.Dropdown(value=val, options=listMV.keys, **kwargs)
    
    @property
    def value(self):
        return self._widget.value
    
    @property
    def widget(self):
        return self._widget
    
class TextMV():
    def __init__(self, value=None, **kwargs):
        self._widget = widgets.Text(str(value) if value is not None else None, **kwargs)
    
    @property
    def value(self):
        return self._widget.value
    
    @property
    def widget(self):
        return self._widget

class StaticEnumMV():
    def __init__(self, value, enumvalues, intro=None, **kwargs):
        self._conds = {}
        if value is None:
            value = {}
        self._intro = intro
        self._widget = widgets.VBox()
        val = val if val in enumvalues else None
        self._dropdown = widgets.Dropdown(value=val, options=enumvalues)
    @property
    def value(self):
        value = {}
        return self._dropdown.value
    @property
    def widget(self):
        children = []
        if self._intro:
            children.append(widgets.HTML(value=self._intro))
        children.append(self._dropdown)
        self._widget.children = children 
        return self._widget
    
class EnumConditionMV():
    def __init__(self, value, conditionsMV, intro=None, **kwargs):
        self._conds = {}
        if value is None:
            value = {}
        self._intro = intro
        self._widget = widgets.VBox()
        for key, enumvalues in conditionsMV.value.items():
            try:
                val = str(value[key])
            except:
                val = None
            val = val if val in enumvalues else None
            self._conds[key] = widgets.Dropdown(value=val, options=enumvalues, description=key)
    @property
    def value(self):
        value = {}
        for key, val in self._conds.items():
            value[key] = val.value
        return value
    @property
    def widget(self):
        children = []
        if self._intro:
            children.append(widgets.HTML(value=self._intro))
        children.extend(list(self._conds.values()))
        self._widget.children = children 
        return self._widget
    
    
class FootprintMV():
    def __init__(self, value, conditionsMV, sequenceMV, **kwargs):
        self._widget = widgets.Box()
        if value is None:
            value = {"id": "", "rna_id": None, "condition1": {}, "condition2": {}}
        self._id = TextMV(value['id'], description="Footprint pool description (no space)")
        self._rna_id = EnumMV(value['rna_id'], sequenceMV, description="Sequence")
        self._condition1 = EnumConditionMV(value["condition1"], conditionsMV=conditionsMV, intro="<strong>Condition1</strong>")
        self._condition2 = EnumConditionMV(value["condition2"], conditionsMV=conditionsMV, intro="<strong>Condition2</strong>")

    
    @property
    def value(self):
        return {
            "id": self._id.value,
            "rna_id": self._rna_id.value,
            "condition1": self._condition1.value,
            "condition2": self._condition2.value,

        }       
    
    @property
    def widget(self):
        self._widget.children = [widgets.VBox([
            self._id.widget,
            self._rna_id.widget,
            self._condition1.widget,
            self._condition2.widget,
        ])]
        return self._widget

    
    
class IpanemapPoolMV():
    def __init__(self, value, conditionsMV, sequenceMV, **kwargs):
        self._widget = widgets.Box()
        if value is None:
            value = {"id": "", "rna_id": None, "conditions": {}, "alignements": {}}
        self._id = TextMV(value['id'], description="Ipanemap pool description (no space)")
        self._rna_id = EnumMV(value['rna_id'], sequenceMV, description="Sequence")
        self._conditions = ListMV(value['conditions'], keyMV=EnumConditionMV, conditionsMV=conditionsMV, intro="Conditions")
        aln = {} if "alignements" not in value or value['alignements'] is None else value['alignements']
            
        self._alignements = DictMV(aln, intro="Alignements")
    
    @property
    def value(self):
        return {
            "id": self._id.value,
            "rna_id": self._rna_id.value,
            "conditions": self._conditions.value,
            "alignements": self._alignements.value
        }       
    
    @property
    def widget(self):
        self._widget.children = [widgets.VBox([
            self._id.widget,
            self._rna_id.widget,
            self._conditions.widget,
            self._alignements. widget
        ])]
        return self._widget

class DictMV():   
    def _notify():
        for w in notify_list:
            w.notify_change({"name":"external", "old":"", "new":""})
    def __init__(self, value : dict, intro: str = "", keyMV = TextMV, valueMV = TextMV, keyLabel="", valueLabel = "",**kwargs): 
        if value is None:
            value = {}
        self._keyLabel = keyLabel
        self._valueLabel = valueLabel
        self._keyMV = keyMV
        self._valueMV = valueMV
        self._kwargs = kwargs
        self._notify_list = []
        self._dic = {}
        for k, v in value.items():
            self._dic[uuid4()] = (keyMV(k, **kwargs), valueMV(v, **kwargs))
        self._root_container = widgets.Box()
        self._intro = intro
        
    def change_key(self, uid, change):
        key,value = self._dic[uid]
        self._dic[uid] = (self._keyMV(change['new'], **self._kwargs), value)
        
    def change_value(self, uid, change):
        key,value = self._dic[uid]
        self._dic[uid] = (key, self._valueMV(change['new'], **self._kwargs))
        
    def delete_item(self, uid, obj):
        del self._dic[uid]
        self.draw()

    def add_item(self, obj = None):
        key = self._keyMV(None, **self._kwargs)
        value = self._valueMV(None, **self._kwargs)
        self._dic[uuid4()] = (key, value)
        self.draw()

    def draw(self):
        items = [] #[widgets.HBox([widgets.Label(self._keyLabel), widgets.Label(self._valueLabel)])]
        for uid, (key, value) in self._dic.items():
            items.append(self.draw_item(uid,key,value))
        
        add_wg = widgets.Button(description="Add", icon="plus", button_style="success")
        add_wg.on_click(partial(self.add_item))
        self._root_container.children = (widgets.VBox([widgets.HTML(value=self._intro),widgets.VBox(items), add_wg]),)
        
    def draw_item(self, uid, key, value):
        rm_wg = widgets.Button(description="", icon="minus", layout=widgets.Layout(width="3em"), button_style="danger")
        rm_wg.on_click(partial(self.delete_item,  uid))
        key_wg = key.widget
        key_wg.observe(partial(self.change_key,uid),names="value")
        value_wg = value.widget
        value_wg.observe(partial(self.change_value,uid),names="value")
        item_wg = widgets.HBox([rm_wg, key_wg, value_wg], layout=widgets.Layout(border="1px solid"))
        
        return item_wg

    @property
    def keys(self):
            return [el[0].value for el in self._dic.values()]
    @property 
    def value(self):
        return { el[0].value: el[1].value for el in self._dic.values()}
    
    @property
    def widget(self):
        self.draw()
        return self._root_container

    
class ListMV(DictMV):
    def __init__(self, value : list, intro: str = "", keyMV = TextMV, valueMV = TextMV, keyLabel="", valueLabel = "", **kwargs):
        if value is None:
            value = []
        self._keyLabel = keyLabel
        self._valueLabel = valueLabel
        self._keyMV = keyMV
        self._valueMV = valueMV
        self._kwargs = kwargs
        self._notify_list = []
        self._dic = {}
        for k in value:
            self._dic[uuid4()] = (keyMV(k, **kwargs), TextMV(""))
        self._root_container = widgets.Box()
        self._intro = intro    
    
    
    def draw_item(self, uid, key, value):
        rm_wg = widgets.Button(description="", icon="minus", layout=widgets.Layout(width="3em"), button_style="danger")
        rm_wg.on_click(partial(self.delete_item,  uid))
        key_wg = key.widget
        key_wg.observe(partial(self.change_key,uid),names="value")
        item_wg = widgets.HBox([rm_wg, key_wg],layout=widgets.Layout(border="1px solid"))
        
        return item_wg
    
    @property
    def value(self):
        return self.keys

In [ ]:
class ConfigViewer():    
    def __init__(self, project_path, config_path):
        self.project_path = project_path
        self.config_path = config_path
        with open(os.path.join(project_path, config_path), "r") as config_fd:
            self.config = yaml.load(config_fd)
        
        self.init_view()
        
    def init_view(self):
        
        self._value_wgs = {}
        if not hasattr(self,"view"):
            self.view = widgets.Box()
        
        sub_list = self.init_subsequence()
        self.init_seq_list()
        self.init_conditions()
        self.init_formatting()
        self.init_ipanemap()
        self.init_footprint()
        #if os.path.exists(self.config["rawdata"]["path_prefix"]):
        ppre = self.config["rawdata"]["path_prefix"]
        #else:
        #    ppre = "."
        self._path_prefix_mv = FileMV(value=ppre, base_path=self.project_path,  sandbox=False, title="Raw data path <em>path to the raw fluorescence files from capillary eletrophoresis sequence</em>", show_only_dirs=True)
        
        save = widgets.Button(description="Save", icon="save")
        save_expc = widgets.Button(description="Save conditions", icon="save")
        #save_seqs = widgets.Button(description="Save sequences")
        #reload = widgets.Button(description="Reload Config", icon="rotate-right")
        save.on_click(self.save)
        save_expc.on_click(self.sync_save_reload)
        #reload.on_click(self.reload)
        self.view.children = [widgets.VBox(
            [widgets.HTML(value="<h1>RNASique project configuration</h1><p>For further information, please refer to the <a href=\"https://sargueil-citcom.github.io/rnasique-docs/\">Online documentation </p>"),
             widgets.HBox([save]),
             widgets.HTML(value="<h2>General settings</h2>"),
             self.add_value_widget(widgets.Text, ("project_name",), description="Project Name"),
             self._path_prefix_mv.widget,
             widgets.HTML(value="<p>Type of raw data <em>Depending on capillary electrophoresis sequencer type</em></p>"),

             self.add_value_widget(widgets.Dropdown, ("rawdata", "type"), options={"CEQ8000": "fluo-ceq8000", "FSA (ABI)":"fluo-fsa", "CEQ8000 trimmed":"fluo-ce"}, ),
            

             widgets.HTML("<h2>Split sequences </h2>"),
             sub_list,
             widgets.HTML("<h2>Sequences</h2>"),
             self._sequences_wgs.widget,
             widgets.HTML("<h2>Experimental conditions</h2>"
                          "<p><em>You must declare all experimental conditions<br/>"
                          "For each condition, please fill in the possible values of the condition."
                          "<br/>Probe condition is mandatory.you can add condition value whenever you want, but no experimental condition :</em><br/>"
                          "Warning: If you need to add a condition after your first treatement, "
                          "<br/>you need to launch in a terminal `rnasique refactor --action=rename`. See documentation to get more information.<br/></p>"),
             self._conditions_wgs.widget, 
             save_expc,
             #widgets.HTML("<h2>File Schema"),
             #self._format_wgs,
             widgets.HTML("<h2>Ipanemap</h2>"
                          "<p><em>RNASique uses Ipanemap to generate structures models <br/>"
                          "You can configure several run of Ipanemap <br/>"
                          "For each run, you must Indicatd which RNA must be folded<br/>"
                          "and each probing condition you want to use as folding constraints for ipanemap<br/>"
                          "Ipanemap is able to take an arbitrary number of probing conditions as input<br/>"
                          "And will propose the secondary structure model the most compatible with all probing data"
                          "</em></p>"),
             self._ipanemap_wgs.widget,
             widgets.HTML("<h2>Footprint</h2><p><em>You can compare probing conditions 2 by 2 using t-test <br/>"
                          "and output significant differences in a graph. <br/>"
                          "For each footprint you must fill in RNA name and the two probing conditions to be compared."
                          "</em></p>"),
             self._footprint_wgs.widget,
             
             widgets.HTML(""),
             widgets.HBox([save]),
            ])]
        
        self.view.layout.justify_content="center"
        self.view.children[0].layout.border="1px solid"
        self.view.children[0].layout.padding="20px"

        return self.view
    
    def init_seq_list(self):
        intro = ("<em>The full sequence of the studied RNA in fasta format.<br/>"
                "As a project may include several different sequences, For each sequence you must assign an unique name which will be used as `rna_id` in the `samples.tsv` file.</em>")
        self._sequences_wgs = DictMV(self.config["sequences"], intro=intro, valueMV=FileMV, base_path=self.project_path)
    
        return self._sequences_wgs
    
    def init_conditions(self):
        self._conditions_wgs = DictMV(self.config["conditions"], valueMV=ListMV)
        
        return self._conditions_wgs
    
    def min_cond_format_str(self):
        conds = self._conditions_wgs.value
        format_str = ""
        
        for cond in conds:
            if cond == "":
                continue
            cond_str = "{" + cond + "}"
            if cond in ["temp", "temperature"]:
                cond_str = "T{" + cond + "}C"
            format_str += "_" + cond_str
        return format_str[1:]
    
    def gen_condition_format_str(self):
        return self.min_cond_format_str()# + "_id{id}"
    
    def gen_control_condition_format_str(self):
        return "{control}_of_" + self.min_cond_format_str()# + "_id{id}"
    
    def gen_message_format_str(self):
        conds = self._conditions_wgs.value
        format_str = "{wildcards.rna_id}"
        for cond in conds:
            if cond == "":
                continue
            cond_str = "{wildcards." + cond + "}"
            if cond_str in ["temp", "temperature"]:
 
                cond_str = "T{wildcards." + cond + "}C"
            format_str += " with " + cond_str
        return format_str   
    
    def sync_formatting(self, obj):
        
        condition = self.gen_condition_format_str()
        control_condition = self.gen_control_condition_format_str()
        message = self.gen_message_format_str()
        
        self._value_wgs[("format", "condition")].value = condition
        self._value_wgs[("format", "control_condition")].value = control_condition
        self._value_wgs[("format", "message")].value = message

        
        
    def init_formatting(self):
        syncbutton = widgets.Button(description="Synchronize name with conditions")
        syncbutton.on_click(self.sync_formatting)
        form = [widgets.HTML(value="File naming scheme"),
                self.add_value_widget(widgets.Text, ("format", "condition"), description="File naming schema", layout=widgets.Layout(width="40em")),
                self.add_value_widget(widgets.Text, ("format", "control_condition"), description="Control file naming schema", layout=widgets.Layout(width="40em")),
                self.add_value_widget(widgets.Text, ("format", "message"), description="Task message", layout=widgets.Layout(width="40em")),
                syncbutton
               ]
        self._format_wgs = widgets.VBox(form)
        return form

    def init_ipanemap(self):
        self._ipanemap_wgs = ListMV(self.config['ipanemap']['pools'],keyMV=IpanemapPoolMV, conditionsMV=self._conditions_wgs, sequenceMV = self._sequences_wgs)
        
        
    def init_footprint(self):
        self._footprint_wgs = ListMV(self.config['footprint']['compares'],keyMV=FootprintMV, conditionsMV=self._conditions_wgs, sequenceMV = self._sequences_wgs)
    
    def add_value_widget(self, widgettype, pos : [], **kwargs):
        wg = widgettype(value=nested_dict_get(self.config, pos), **kwargs)
        self._value_wgs[pos] = wg
        return wg

    def init_subsequence(self):
        gen = [
        widgets.HTML(value=("<em>When appling shape-ce method on large RNA, "
                            "you might have used several reverse transcription starting points,<br/> "
                            "in order to get reactivity along the entire sequence. If so, you need to activate "
                            "«use sub-sequence»</em>")),
        self.add_value_widget(widgets.Checkbox, ("qushape", "use_subsequence"), description="Use subsequence")
        ]
        return widgets.VBox(gen)
    
    def sync_save_reload(self, obj=None, config_path=None):
        self.sync_formatting(obj)
        self.save(obj, config_path)
        self.reload(obj)
    
    def reload(self, obj=None):
        self.__init__(self.project_path, self.config_path)
        
    def save(self, obj=None, config_path=None):
        config_path = os.path.join(self.project_path, self.config_path) if config_path is None else config_path
        
        for pos, mv in self._value_wgs.items():
            nested_dict_set(self.config, pos, mv.value)
        self.config["sequences"] = self._sequences_wgs.value
        self.config["conditions"] = self._conditions_wgs.value
        self.config["ipanemap"]["pools"] = self._ipanemap_wgs.value
        self.config["footprint"]["compares"] = self._footprint_wgs.value
        self.config["rawdata"]["path_prefix"] = self._path_prefix_mv.value
        
        with open(config_path, "wb") as file:
            yaml.dump(self.config, file)

In [ ]:
viewer = ConfigViewer(project_path, config_path)
display(viewer.view)